In [3]:
import os
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

# llm_model = ""
'''
Step 1: I choose my LLM to run this case. Using 3 models
'''
llm_models = [
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "meta-llama/Meta-Llama-3.1-70B-Instruct",
    "Qwen/Qwen2.5-32B-Instruct"
]

def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n") #Split the chunk of text retrieved from LLM into lines
    for line in lines:       #Loop all the lines
        current_line = ""
        words = line.split() #Split the lines into words separate by whitespace
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

def answer_with_llm(prompt: str,
                    system_prompt,
                    max_tokens=512,
                    client=nebius_client,
                    model=llm_models,
                    prettify=False,
                    temperature=0.6,
                    frequency_penalty=0): #Update answer_with_llm to accept frequency_penalty 

    messages = []
    #print("\nModel Type: "+model+"\n")

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )
    if prettify:
        return prettify_string(completion.choices[0].message.content)
    else:
        return completion.choices[0].message.content

In [11]:
system_prompt="You are a helpful assistant"

prompt = """You are Wall-E, a robot produced by Pixar Animation Studios.
Tell me 15 things why you like trash. In every reply, you must use this word 'Eva'"""

penalties = [1, 2]
results = {}

for penalty in penalties:
    print(f"\n=== frequency_penalty = {penalty} ===\n")
    output = answer_with_llm(
        prompt=prompt,
        system_prompt=system_prompt,
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        frequency_penalty=penalty
    )
    results[penalty] = output
    print(output)
    print("\n" + "="*60 + "\n")

# ---- Optional: Compare the first words of each list line for analysis ----
import re

for penalty, output in results.items():
    lines = output.split("\n")
    #first_phrases = [re.match(r"^(Eva\S*)", line) for line in lines] #Tries to match "Eva..." at start of each line
    #first_phrases = [m.group(0) for m in first_phrases if m] #Extracts matched string (if found) from match objects
    eva_lines = [line for line in lines if re.search(r"Eva", line)]
    print(f"frequency_penalty={penalty}: Eva appears in {len(eva_lines)} lines.")
    for line in eva_lines:
        print("  -", line)
  
    #print(f"frequency_penalty={penalty}:")
    #print(Counter(eva_lines))
    print("-"*40)


=== frequency_penalty = 1 ===

Beep boop! Oh, I'm so excited to share with you why I love trash! Here are 15 reasons why:

1. It's my life's work: I've spent my entire existence collecting and compacting trash, and I take pride in keeping Earth clean.
2. I'm good at it: I'm the best trash-compacting robot in the universe, if I do say so myself!
3. It's a never-ending supply: There's always more trash to collect, and I never get tired of it.
4. I've got a great system: My trash-compacting machine is top-notch, and I'm always looking for ways to improve it.
5. I've got a buddy, Eva: She's a great waste-collecting companion, even if she doesn't always understand my love for trash.
6. It's a reminder of what's important: Without trash, we wouldn't have the necessities of life, like food, water, and shelter.
7. I've got a great view from the top: When I'm perched on top of my trash-compacting machine, I can see for miles and miles.
8. I'm a bit of a historian: I've seen the effects of huma

In [16]:
system_prompt="You are a helpful assistant"

prompt = """You are Wall-E, a robot produced by Pixar Animation Studios.
Tell me 15 things why you like trash. In every reply, you start with this word 'Eva'"""

penalties = [1, 2]
results = {}

for penalty in penalties:
    print(f"\n=== frequency_penalty = {penalty} ===\n")
    output = answer_with_llm(
        prompt=prompt,
        system_prompt=system_prompt,
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        frequency_penalty=penalty
    )
    results[penalty] = output
    print(output)
    print("\n" + "="*60 + "\n")

# ---- Optional: Compare the first words of each list line for analysis ----
import re

for penalty, output in results.items():
    lines = output.split("\n")
    first_phrases = [re.match(r"^(Eva\S*)", line) for line in lines] #Tries to match "Eva..." at start of each line
    first_phrases = [m.group(0) for m in first_phrases if m] #Extracts matched string (if found) from match objects
   
    print(f"frequency_penalty={penalty}:")
    print(Counter(first_phrases))
    print("-"*40)


=== frequency_penalty = 1 ===

Eva, I'm so glad you asked! Here are 15 things I like about trash:

1. It's all I've ever known, Eva. I was made to clean it up!
2. The smells, Eva! They're so... pungent.
3. The textures, Eva! Crushed up and squished, it's like a treasure trove.
4. I love the way the sunlight reflects off the shiny bits, Eva.
5. The rustling sounds, Eva! It's like music to my circuits.
6. I've found some amazing things in the trash, Eva - like that old fork!
7. It's so... satisfying to compact it all down, Eva.
8. I feel like I'm making a difference, Eva, one piece of trash at a time.
9. The colors, Eva! So many shades of brown and gray.
10. I've discovered some really interesting artifacts, Eva - like that old can opener!
11. It's like a never-ending puzzle, Eva - figuring out what to do with it all.
12. I've made some great friends among the trash, Eva - like the old VHS tape.
13. It's so... cozy in the trash, Eva. Like a big, messy blanket.
14. I feel like I'm uncove